In [ ]:
%load_ext autoreloadfrom pycaret.classification import load_model, predict_model
%autoreload 2
import pandas as pd
import pickle  
import os
from pycaret.classification import load_model, predict_model
import datetime
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading different datasets

data_pool = {
        'pq': pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='quiruelas', engine='openpyxl'),
        'vdh': pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='v_higueras', engine='openpyxl'),
        'da': pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Alberite', engine='openpyxl'),
        'pa':pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Paternanbidea', engine='openpyxl'),
        'cg':pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Can_Gambus', engine='openpyxl'),
        'cs':pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='CatalonianSites', engine='openpyxl'),
        'fs': pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='FrenchSites', engine='openpyxl')
 }

In [ ]:
data_pool = {}
        
        # Load data from Excel sheets with proper error handling
        try:
            excel_path = '/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx'
            sheets = ['quiruelas', 'v_higueras', 'Alberite', 'Paternanbidea', 
                      'Can_Gambus', 'CatalonianSites', 'FrenchSites']
            
            # Load one by one with error handling
            for sheet, key in zip(sheets, ['pq', 'vdh', 'da', 'pa', 'cg', 'cs', 'fs']):
                try:
                    df = pd.read_excel(excel_path, sheet_name=sheet, engine='openpyxl')
                    

                    
                    # Store the processed dataframe
                    data_pool[key] = df
                    logger.info(f"Loaded sheet '{sheet}' with {len(df)} rows")
                except Exception as e:
                    logger.error(f"Error loading sheet '{sheet}': {str(e)}")
        except Exception as e:
            logger.error(f"Error accessing Excel file: {str(e)}")
            logger.info("Using empty data pool due to data loading errors")

In [ ]:
model_pool = {}
        model_paths = {
            'full_model': '/home/dsg/VORTEX_FINAL/PRODUCTION/notebooks/models/final_model_direct',
            'VdHModel': '/home/dsg/VORTEX_FINAL/PRODUCTION/models/20250227_VdHSpecific',
            'PQModel': '/home/dsg/VORTEX_FINAL/PRODUCTION/models/20250227_QuiruelasSpecific',
            'FrenchModel': '/home/dsg/VORTEX_FINAL/PRODUCTION/models/20250227_FrenchSpecific'
        }

       
        
        for model_name, model_path in model_paths.items():
            try:
                # Intentar primero con load_model de PyCaret
                try:
                    model = load_model(model_path)
                    logger.info(f"Loaded model '{model_name}' using PyCaret from {model_path}")
                except Exception as e:
                    logger.warning(f"Could not load with PyCaret: {str(e)}")

In [ ]:
def prediction_function(model_path, excel_path, exclude_columns=None, output_column='predictions'):
    """
    Loads an Excel or CSV file,
    makes predictions using a pre-trained model,
    and saves the results in a new Excel or CSV file.

    :param model_path: Ruta al archivo del modelo 
    :param excel_path: Ruta al archivo con los datos de entrada
    :param exclude_columns: Lista de columnas a excluir como entrada (opcional)
    :param output_column: Nombre de la columna para las predicciones (por defecto: 'predictions')
    :return: Ruta al archivo de salida con las predicciones
    """
    # Loading  the model usando PyCaret
    model = load_model(model_path)

    # Cargar los datos
    sheet = ['quiruelas', 'v_higueras', 'Alberite', 'Paternanbidea', 
                      'Can_Gambus', 'CatalonianSites', 'FrenchSites']
    
    df = pd.read_excel(excel_path, sheet_name=sheet)

    # Excluir columnas si se especifican
    if exclude_columns:
        df = df.drop(columns=[col for col in exclude_columns], errors='ignore')

    # Hacer predicciones usando PyCaret
    predictions = predict_model(model, data=df,raw_score=True)
    
    # El resultado de predict_model ya incluye las predicciones, 
    # generalmente en una columna llamada 'Label' o similar
    # Renombrar la columna de predicciones si es necesario
    if 'Label' in predictions.columns:
        predictions = predictions.rename(columns={'Label': output_column})
    elif 'prediction_label' in predictions.columns:
        predictions = predictions.rename(columns={'prediction_label': output_column})
    else:
        # Si la columna de predicciones tiene otro nombre, ajusta esto según sea necesario
        pass

   
    # Create name for the output file
    base_name = os.path.splitext(excel_path)[0]
    current_date = datetime.datetime.now().strftime("%Y%m%d")
    output_path = f"{base_name}_{current_date}_predictions.xlsx"
   

    # save results
    predictions.to_excel(output_path, index=False)
    predictions.to_csv(output_path, index=False)

    return output_path

In [ ]:

# Usage:
model_path = '/home/dsg/VORTEX_FINAL/PRODUCTION/models/20250227_VdHSpecific'
excel_path = '/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx'
sheet_name='v_higueras'
exclude_columns = [] 

try:
    output_file = prediction_function(model_path, excel_path, exclude_columns)
    print(f"Predictions saved in: {output_file}")
except Exception as e:
    print(f"Error processing the file: {str(e)}")